# SDK End-to-End Example

Create a workflow, register it and execute it using the client.

In [ ]:
import os
from dotenv import load_dotenv
from kubiya_workflow_sdk import KubiyaClient, workflow

# Load environment variables
load_dotenv()

# Get API key
api_key = os.getenv("KUBIYA_API_KEY")
if not api_key:
    raise ValueError("KUBIYA_API_KEY is required. Set it in your .env file")

# Create a complete end-to-end example workflow
deployment_workflow = (
    workflow("app-deployment")
    .description("End-to-end application deployment workflow")
    .params(
        app_name="${APP_NAME:-myapp}",
        version="${VERSION:-latest}",
        environment="${ENV:-staging}"
    )
    # Step 1: Pre-deployment checks
    .step("pre-check", """
        echo "🔍 Running pre-deployment checks for ${app_name}:${version}"
        echo "Target environment: ${environment}"
    """)
    # Step 2: Build and test
    .step("build", "echo '🔨 Building ${app_name}:${version}'")
    .step("test", "echo '🧪 Running tests...' && sleep 2 && echo '✅ Tests passed!'")
    # Step 3: Deploy
    .step("deploy", """
        echo "🚀 Deploying ${app_name}:${version} to ${environment}"
        echo "📦 Pulling image..."
        echo "🔄 Rolling update in progress..."
        sleep 3
        echo "✅ Deployment successful!"
    """)
    # Step 4: Verify
    .step("verify", "echo '🏥 Health check passed!'")
)

# Initialize client with real API
client = KubiyaClient(api_key=api_key)

print("✅ SDK End-to-End Example Ready")
print(f"📋 Workflow: {deployment_workflow.data['name']}")
print(f"🔗 API: Connected to Kubiya API")

In [ ]:
# Execute the workflow with the client
print("🚀 Executing end-to-end workflow with real API...")
print("=" * 60)

from kubiya_workflow_sdk.execution import execute_workflow_logged, LogLevel

# Set custom parameters
params = {
    "app_name": "demo-app",
    "version": "v2.0.1",
    "environment": "production"
}

print(f"\n📝 Parameters:")
for key, value in params.items():
    print(f"   {key}: {value}")

try:
    print("\n🔄 Starting execution...")
    event_count = 0
    
    for event in execute_workflow_logged(
        workflow_definition=deployment_workflow.to_dict(),
        api_key=api_key,
        parameters=params,
        log_level=LogLevel.NORMAL
    ):
        event_count += 1
        # Events are printed by the logger
    
    if event_count > 0:
        print(f"\n✅ Workflow executed successfully!")
        print(f"   Total events: {event_count}")
    else:
        print("\n⚠️  No events received")
        
except Exception as e:
    print(f"\n❌ Execution error: {str(e)}")
    print("\nNote: Ensure your Kubiya organization has active runners")
    
print("\n✅ End-to-end example complete!")